In [1]:
import time
import telebot
import sched
import datetime
import dateparser
import logging
import threading
import uuid

logging_level = logging.DEBUG
logger = logging.getLogger("rmndrBot" if __name__ == "__main__" else __name__)
c_handler = logging.StreamHandler()
logger.setLevel(logging_level)
c_handler.setLevel(logging_level)
c_format = logging.Formatter('%(asctime)s - %(threadName)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
logger.addHandler(c_handler)

In [2]:
with open('apikey.txt', 'r') as f:
    apikey = f.readline()

contact_data = 'example@example.com'
bot = telebot.TeleBot(apikey)
tasks = {}
tasks_mutex_dict = {}
user_threads = {}
# s = sched.scheduler(time.time, time.sleep)


from dateparser.date import DateDataParser
class MiracleDateParser(DateDataParser):
    def __init__(self):
        super().__init__(languages=['en', 'ru'])
    def get_date_data(self, date_string, date_formats=None):
        parsed = super().get_date_data(date_string, date_formats)
        if not parsed['date_obj']:
            return None
        return parsed
    
SupercoolMiracleDateParserInstance = MiracleDateParser()


def dummy_time():
    dt = datetime.datetime.now()
    td = datetime.timedelta(seconds=5)
    dt += td
    return dt   

def message_to_user(user_id, message):
    bot.send_message(user_id, message)

def new_task(user_id, task):
    if user_id in tasks:
        tasks[user_id].append(task)
        tasks_mutex_dict[user_id].acquire() # synchronized
        tasks[user_id] = sorted(tasks[user_id], key=lambda task: task.scheduled_time)
        tasks_mutex_dict[user_id].release()
    else:
        tasks[user_id] = [task]
        tasks_mutex_dict[user_id] = threading.Lock()
        new_user(user_id)
    
def new_user(user_id, refresh_interval=5):
    new_thread = User_Thread(user_id, refresh_interval)
    user_threads[user_id] = (new_thread, True)
    logger.info("New user, id: %d!" % user_id)
    new_thread.start()
#     message_to_user(user_id, "Welcome to ReminderBot. We hope you will enjoy using it!")

class Task(object):
    def __init__(self, user_id, message_text, parser):
        self.parser = parser
        self.scheduled_time, self.content = self.parse_time_and_content(message_text)
        self.creation_timestamp = datetime.datetime.now()
        self.user_id = user_id
        self.id = uuid.uuid4()
        logger.debug("New task id %s for user %d scheduled for %s." % (str(id), self.user_id, str(self.scheduled_time)))
    
    def parse_time_and_content(self, message_text):
        start_time = len(message_text.split()) if len(message_text.split()) < 6 else 5
        for i in range(start_time):
            parsed_time = self.parser.get_date_data(' '.join(message_text.split()[0:i]))
            if parsed_time:
                scheduled_time = parsed_time['date_obj']
                text = ' '.join(message_text.split()[i:])
            else:
                continue
        return scheduled_time, text
    
    def execute(self):
        message_to_user(self.user_id, self.content)
        
# one per user_id
class User_Thread(threading.Thread):
    def __init__(self, user_id, refresh_interval):
        threading.Thread.__init__(self)
        self.user_id = user_id
        self.tasks_list = tasks[user_id]
        self.tasks_mutex = tasks_mutex_dict[user_id]
        self.refresh_interval = refresh_interval
        self.scheduler = sched.scheduler()
        
    def run(self):
        logger.info("thread started!")
        while user_threads[self.user_id][1]:
            self.schedule()
        logger.info("thread for user id %d finished." % user_id)
        
    def schedule(self):
        self.tasks_mutex.acquire() # synchronized
        self.tasks_list = tasks[self.user_id]
        if self.tasks_list:
            logger.debug("Tasks_list: %s" % str(self.tasks_list))
            time_left = self.tasks_list[0].scheduled_time - datetime.datetime.now()
            if time_left.total_seconds() < self.refresh_interval:
                interval = time_left.total_seconds()
            else:
                interval = self.refresh_interval
        else:
            interval = self.refresh_interval
        self.tasks_mutex.release()
        self.scheduler.enter(interval, 1, self.clear_tasks)
#         logger.debug("Scheduler queue: %s" % str(self.scheduler.queue))
        self.scheduler.run()
        return
        
    def clear_tasks(self):
        self.tasks_mutex.acquire() # synchronized
        if(self.tasks_list):
            if self.tasks_list[0].scheduled_time < datetime.datetime.now():
                logger.debug("Popping task scheduled for %s for user id %d" % (str(self.tasks_list[0].scheduled_time), self.user_id))
                task = self.tasks_list.pop(0)
                task.execute()
        self.tasks_mutex.release()

@bot.message_handler(commands=['help',])
def send_welcome(message):
    instructions  = """Hi!\nSend me a message like \"Tomorrow 18:00 Dinner with Jack\" and I'll remind you at the time you specified!\nPlease report any problems at %s""" % contact_data
    bot.reply_to(message, instructions)
    
@bot.message_handler(commands=['tasks',])
def display_tasks(message):
    user_id = message.from_user.id
    tasks_for_user = tasks[user_id]
    result = [str(task.scheduled_time) + ' ' + str(task.content) for task in tasks_for_user]
    message_to_user(user_id, '\n'.join(result))
        
@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    in_text = message.text
    user_id = message.from_user.id
    logger.debug("Received message %s from user_id %d." % (in_text, user_id))
    t = Task(user_id, in_text,SupercoolMiracleDateParserInstance)
    new_task(user_id, t)
    message_to_user(user_id, "New reminder scheduled for %s." % str(t.scheduled_time))
    logger.debug("Task queue for user %d: %s." % (user_id, str([task.id for task in tasks[user_id]])))

bot.polling(none_stop=True, interval=1)

2019-04-07 19:22:50,497 - WorkerThread1 - DEBUG - Received message Tomorrow wake up from user_id 220340386.
2019-04-07 19:22:51,341 - WorkerThread1 - DEBUG - New task id <built-in function id> for user 220340386 scheduled for 2019-04-08 19:22:51.287854.
2019-04-07 19:22:51,345 - WorkerThread1 - INFO - New user, id: 220340386!
2019-04-07 19:22:51,350 - Thread-6 - INFO - thread started!
2019-04-07 19:22:51,388 - Thread-6 - DEBUG - Tasks_list: [<__main__.Task object at 0x0000007EE6A2A940>]
2019-04-07 19:22:52,341 - WorkerThread1 - DEBUG - Task queue for user 220340386: [UUID('e4462865-44dd-439b-8d27-23daa2c8617a')].
2019-04-07 19:22:56,396 - Thread-6 - DEBUG - Tasks_list: [<__main__.Task object at 0x0000007EE6A2A940>]
2019-04-07 19:23:01,400 - Thread-6 - DEBUG - Tasks_list: [<__main__.Task object at 0x0000007EE6A2A940>]
2019-04-07 19:23:06,404 - Thread-6 - DEBUG - Tasks_list: [<__main__.Task object at 0x0000007EE6A2A940>]
2019-04-07 19:23:11,408 - Thread-6 - DEBUG - Tasks_list: [<__main__

In [ ]:
#heapq experiments: probably should use this instead of list sorted after each append call
# import heapq

# h = []
# heapq.heappush(h, 4)
# heapq.heappush(h, 2)
# heapq.heappush(h, 1)
# print(type(h))
# print(h)
# print(list(h))
# print(heapq.heappop(h))
# print(heapq.heappop(h))

In [4]:
def parse_time_and_content(message_text):
    import dateparser
    scheduled_time = dateparser.parse(' '.join(message_text.split()[0:2]))
    content = ' '.join(message_text.split()[2:])
    return scheduled_time, content

a, b = parse_time_and_content("Tomorrow say hi")
print(b)

None


In [17]:
import dateparser
from dateparser.search import search_dates


in_text = "Tomorrow say hi"
# something is wrong with word do!!

def ParseTime(in_text):
    
    for: # с конца
    if : panic()
            
#     if not parsed_time and not text:
#         dates = search_dates(in_text) 
#         if len(dates)==1:
#             parsed_time=dates[0][0]
#             text=in_text.replace(dates[0][0],'')
#         else: 
#             #State-of-the-art AI that asks user which date to choose 
#             print("More than 1 date")
#             print(dates)
#             for i in dates:
#                 continue
    return parsed_time,text
print(ParseTime(in_text))

(datetime.datetime(2019, 4, 8, 16, 44, 31, 885850), 'say hi')


In [58]:
import datetime
# print( dateparser.parse("tuesday"))
datetime.datetime.strftime(search_dates("tomorrow next now", settings={'PREFER_DATES_FROM': 'future'})[0][1], "%Y-%m-%d %H:%M:%S")

'2019-04-08 17:19:34'

In [113]:
from dateparser.date import DateDataParser
class MiracleDateParser(DateDataParser):
    def __init__(self):
        super().__init__(languages=['en', 'ru'])
    def get_date_data(self, date_string, date_formats=None):
        parsed = super().get_date_data(date_string, date_formats)
        if not parsed['date_obj']:
            raise OverflowError("Wtf")
        return parsed
    
a = MiracleDateParser()
# try:
# strin = "next year at 6am"
# case(asd):
#     "next"
#     "at"
#     "last"
#     "this"
x = a.get_date_data("завтра")
# except OverflowError:
#     print("yay")
x

{'date_obj': datetime.datetime(2019, 4, 8, 18, 52, 38, 180034),
 'locale': 'ru',
 'period': 'day'}

In [94]:
x

{'date_obj': datetime.datetime(2019, 4, 7, 21, 0),
 'locale': 'en',
 'period': 'day'}